In [19]:
import pandas as pd
from bert_serving.client import BertClient
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation 

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

bc = BertClient()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jzheadley/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jzheadley/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jzheadley/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
s = pd.read_json('../data/clean_embedded.json',orient='records')

In [21]:
print(s)

       Unnamed: 0   country  \
0               0     Italy   
1               1  Portugal   
2               2        US   
3               3        US   
4               4        US   
...           ...       ...   
24995       24995        US   
24996       24996     Chile   
24997       24997    France   
24998       24998    France   
24999       24999        US   

                                             description  \
0      Aromas include tropical fruit, broom, brimston...   
1      This is ripe and fruity, a wine that is smooth...   
2      Tart and snappy, the flavors of lime flesh and...   
3      Pineapple rind, lemon pith and orange blossom ...   
4      Much like the regular bottling from 2012, this...   
...                                                  ...   
24995  This blend of 46% Cabernet Sauvignon, 34% Syra...   
24996  A toasty but green-smelling bouquet mixes scen...   
24997  This wine is all bright cherry fruit with juic...   
24998  This is an attractiv

In [22]:
s['id']=s['Unnamed: 0']

In [23]:
def clean_data(desc):
    words = stopwords.words('english')
    lower = " ".join([w for w in desc.lower().split() if not w in words])
    punct = ''.join(ch for ch in lower if ch not in punctuation)
    wordnet_lemmatizer = WordNetLemmatizer()

    word_tokens = nltk.word_tokenize(punct)
    lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]

    word_joined = " ".join(lemmatized_word)
    
    return word_joined

In [39]:
from scipy import spatial

cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)
similarities = []

# inputted_sentence = ["dry with a fruity aftertaste"] # this inputted sentence needs to get vectorized
inputted_sentence = "floral essence " # this inputted sentence needs to get vectorized
cmp_sent = bc.encode([clean_data(inputted_sentence)]) # vectorizing the senetence
for index, row in s.iterrows():
    similarity = cosine_similarity(cmp_sent,row['embeddings'])
    similarities.append((row['id'],row['title'],similarity))
# for sentences in s["embeddings"]:
#     similarity = cosine_similarity(cmp_sent, sentences)
#     print(similarity)
#     similarities.append((sentences,similarity))
    
similarities = sorted(similarities, key=lambda item: -item[2])

In [40]:
# print(cmp_sent)
print(similarities[-5:])
best = similarities[len(similarities)-1]
print(best)


[(16292, 'Aia Vecchia 2010 Sorugo  (Bolgheri Superiore)', 0.7024512664442345), (15425, 'Westwood 2015 Annadel Gap Vineyard White (Sonoma Valley)', 0.7023228458340438), (19552, 'Renieri 2013 Re di Renieri Red (Toscana)', 0.7021261921424243), (6621, "Le P'tit Paysan 2011 Pierre's Pirouette Rosé of Mourvèdre (San Benito County)", 0.6645734147573127), (18978, 'Le Colture NV Cartizze  (Prosecco Superiore di Cartizze)', 0.6413750614102952)]
(18978, 'Le Colture NV Cartizze  (Prosecco Superiore di Cartizze)', 0.6413750614102952)


In [41]:
best_instance = s.iloc[best[0]]
print(best_instance)

Unnamed: 0                                                           18978
country                                                              Italy
description              This is a very feminine sparkler with a beauti...
designation                                                       Cartizze
points                                                                  89
price                                                                   38
province                                                            Veneto
region_1                                    Prosecco Superiore di Cartizze
region_2                                                              None
taster_name                                                           None
taster_twitter_handle                                                 None
title                    Le Colture NV Cartizze  (Prosecco Superiore di...
variety                                                           Prosecco
winery                   

In [43]:
# print([w[0] for w in similarities[:10]])
print(len(best_instance['description']))
print(best_instance['description'])

201
This is a very feminine sparkler with a beautiful bouquet of fresh white rose and jasmine. It has a delicate, pulpy consistency and would pair well with zucchini flowers stuffed with mozzarella cheese.
